In [1]:
#****************************************************************************
# (C) Cloudera, Inc. 2020-2023
#  All rights reserved.
#
#  Applicable Open Source License: GNU Affero General Public License v3.0
#
#  NOTE: Cloudera open source products are modular software products
#  made up of hundreds of individual components, each of which was
#  individually copyrighted.  Each Cloudera open source product is a
#  collective work under U.S. Copyright Law. Your license to use the
#  collective work is as provided in your written agreement with
#  Cloudera.  Used apart from the collective work, this file is
#  licensed for your use pursuant to the open source license
#  identified above.
#
#  This code is provided to you pursuant a written agreement with
#  (i) Cloudera, Inc. or (ii) a third-party authorized to distribute
#  this code. If you do not have a written agreement with Cloudera nor
#  with an authorized and properly licensed third party, you do not
#  have any rights to access nor to use this code.
#
#  Absent a written agreement with Cloudera, Inc. (“Cloudera”) to the
#  contrary, A) CLOUDERA PROVIDES THIS CODE TO YOU WITHOUT WARRANTIES OF ANY
#  KIND; (B) CLOUDERA DISCLAIMS ANY AND ALL EXPRESS AND IMPLIED
#  WARRANTIES WITH RESPECT TO THIS CODE, INCLUDING BUT NOT LIMITED TO
#  IMPLIED WARRANTIES OF TITLE, NON-INFRINGEMENT, MERCHANTABILITY AND
#  FITNESS FOR A PARTICULAR PURPOSE; (C) CLOUDERA IS NOT LIABLE TO YOU,
#  AND WILL NOT DEFEND, INDEMNIFY, NOR HOLD YOU HARMLESS FOR ANY CLAIMS
#  ARISING FROM OR RELATED TO THE CODE; AND (D)WITH RESPECT TO YOUR EXERCISE
#  OF ANY RIGHTS GRANTED TO YOU FOR THE CODE, CLOUDERA IS NOT LIABLE FOR ANY
#  DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, PUNITIVE OR
#  CONSEQUENTIAL DAMAGES INCLUDING, BUT NOT LIMITED TO, DAMAGES
#  RELATED TO LOST REVENUE, LOST PROFITS, LOSS OF INCOME, LOSS OF
#  BUSINESS ADVANTAGE OR UNAVAILABILITY, OR LOSS OR CORRUPTION OF
#  DATA.
#
# #  Author(s): Paul de Fusco
#***************************************************************************/

In [1]:
!pip3 install open-inference-openapi 
#!pip3 install httpx uvicorn

  Using cached open_inference_openapi-2.0.0.1-py3-none-any.whl.metadata (5.6 kB)
Using cached open_inference_openapi-2.0.0.1-py3-none-any.whl (29 kB)


### Deploy the Model to CAI Inference Service

In [6]:
myToken = ""

In [7]:
import json
import httpx

#JWT = json.load(open("/tmp/jwt"))["access_token"]
JWT = myToken
headers = {'Authorization': 'Bearer ' + JWT,
           'Content-Type': 'application/json'}

httpx_client = httpx.Client(headers=headers)

In [8]:
CAII_DOMAIN_URL = 'https://ml-cfe35b45-935.rapids-d.a465-9q4k.cloudera.site'

deploy_payload = {
 "namespace": "serving-default",
 "name": "Paul Onnx Xgboost",
 "source": {
   "registry_source": {
      "model_id": "zjrw-n7gg-6v8g-m2ic",
      "version": 1
    }
  },
 "resources": {
    "req_cpu": "2",
    "req_memory": "8Gi"
  },
  "api_standard": "oip",
  "has_chat_template": False,
  "metric_format": "triton",
  "task": "INFERENCE",
  "instance_type": "m5.4xlarge",
 "autoscaling": {
    "min_replicas": "1",
    "max_replicas": "100"
  }
}

r = httpx_client.post(CAII_DOMAIN_URL+'/api/v1alpha1/deployEndpoint', json=deploy_payload)
r

<Response [200 OK]>

In [9]:
CAII_DOMAIN_URL = 'https://ml-cfe35b45-935.rapids-d.a465-9q4k.cloudera.site'
# Wait for the endpoint to be ready
describe_payload = {
    "namespace": "serving-default",
    "name": "XGBoost Fraud Detector"
}

r = httpx_client.post(CAII_DOMAIN_URL+'/api/v1alpha1/describeEndpoint', json=describe_payload)
status = r.json()['status']
response = json.dumps(r.json(), indent=2)
print(response)

# Want target_model_state to get to 'Loaded'
status['target_model_state']

KeyError: 'status'

In [10]:
r

<Response [404 Not Found]>